[2025-09-05 Fiddler](https://thefiddler.substack.com/p/how-low-or-high-can-you-go)
--------------------

Fiddler
-------
Without loss of generality, assume $x_1 > 1/2$ and normalize with a factor of 2.

1. The probability that $x_2 < 1/2$ is $1/2$.
2. The probability that $1/2 < x_2 < x_1$ is $x_1-1/2$.
3. The probability that $x_2 > x_1$ is $1-x_1$.

The probability of winning a second point in the first case is $1-x_2$.

The probability of winning a second point in the second case is $x_2$.

The game is over in the third case.

Integrating over $x_1$ and $x_2$,

$$ p_2 = 2\int_{1/2}^1 dx_1\,\left(\int_0^{1/2} dx_2\,(1-x_2) + \int_{1/2}^{x_1} dx_2\, x_2\right) = 13/24 $$

In [1]:
x1,x2 = var('x1,x2')

p2 = 2*integral(integral(1-x2, x2, 0, 1/2)
                + integral(x2, x2, 1/2, x1),
                x1, 1/2, 1)
p2, numerical_approx(p2)

(13/24, 0.541666666666667)

[Numerical simulations](20250905.go) agree

    $ go run 20250905.go
    43326720/80000000=0.541584